In [ ]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression

In [2]:
train=pd.read_csv("twitter_train.csv")
test=pd.read_csv("twitter_test.csv")

In [3]:
training=train.to_numpy()
testing=test.to_numpy()

In [4]:
train_data=training[:,6:8]
test_data=testing[:,5:7]

In [5]:
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
lemmatizer=WordNetLemmatizer()

In [6]:
X_train=train_data[:,1]
Y_train=train_data[:,0]

X_test=test_data[:,1]
Y_test=test_data[:,0]

In [7]:
stops=stopwords.words('english')
import string 
punctuations=list(string.punctuation)
stops+=punctuations

In [8]:
x_tr=[w for w in X_train if not w in stops]
y_tr=[w for w in Y_train if not w in stops]
x_ts=[w for w in X_test if not w in stops]
y_ts=[w for w in Y_test if not w in stops]

In [9]:
st=PorterStemmer()
xtr=[st.stem(w) for w in x_tr]
xts=[st.stem(w) for w in x_ts]

In [10]:
train_document=[]
j=0
for i in (xtr):
    if y_tr[j]==0:
        train_document.append((i,"neg"))
        j+=1
    else:
        train_document.append((i,"tru"))
        j+=1

In [11]:
test_document=[]
j=0
for i in (xts):
    if y_tr[j]==0:
        test_document.append((i,"neg"))
        j+=1
    else:
        test_document.append((i,"pos"))
        j+=1

In [12]:
def sample_tag_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('v'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [13]:
from nltk.corpus import wordnet
def clean_reviews(words):
    output_word=[]
    for w in words:
        if w.lower() not in stops:
            pos=pos_tag([w])
            clean=lemmatizer.lemmatize(w,pos=sample_tag_pos(pos[0][1]))
            output_word.append(clean.lower())
    return output_word

In [14]:
from nltk import pos_tag
train_da=[]
test_da=[]
for i in range(len(train_document)):
    a=train_document[i][0]
    c=word_tokenize(a)
    b=[w for w in c if not w in stops]
    train_da.append((b,train_document[i][1]))
for i in range(len(test_document)):
    a=test_document[i][0]
    c=word_tokenize(a)
    b=[w for w in c if not w in stops]
    test_da.append((b,test_document[i][1]))


In [15]:
train_documents=[(clean_reviews(document1),cat) for document1, cat in train_da]
test_documents=[(clean_reviews(document1),cat) for document1, cat in test_da]

In [16]:
all_word=[]
for doc in train_documents:
    all_word+=doc[0]   

In [17]:
import nltk 
frequency=nltk.FreqDist(all_word)  ## how many words are common and how much?
common=frequency.most_common(300)          ## here print only 15 highest common words
features=[i[0] for i in common]

In [18]:
def get_word_doc(word):
    current_feature={}
    words_set=set(word)
    for w in features:
        current_feature[w]=w not in words_set
    return current_feature

In [19]:
training_final_data=[(get_word_doc(doc),category) for doc, category in train_documents]
testing_final_data=[(get_word_doc(doc),category) for doc, category in test_documents]

In [20]:
from nltk import NaiveBayesClassifier
classifier=NaiveBayesClassifier.train(training_final_data)
nltk.classify.accuracy(classifier,testing_final_data)

0.9322404371584699

In [22]:
from sklearn.svm import SVC
from nltk.classify.scikitlearn import SklearnClassifier
svc=SVC()
sklearn_classifier=SklearnClassifier(svc)

In [23]:
sklearn_classifier.train(training_final_data)
nltk.classify.accuracy(sklearn_classifier,testing_final_data)

C:\Users\azam ali\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


0.9510928961748634

In [106]:
from sklearn.ensemble import RandomForestClassifier
rfd=RandomForestClassifier()
classifier_sklearner=SklearnClassifier(rfd)

In [107]:
classifier_sklearner.train(training_final_data)


C:\Users\azam ali\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


<SklearnClassifier(RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False))>

In [108]:
nltk.classify.accuracy(classifier_sklearner,testing_final_data)

0.9489071038251367